## Language interpreter

We'll cover the following topics in this notebook:
- Expression interpreter with variables
- Interpreting let-bindings and functions
- Interpreting recursive functions

### Retake: calculator

In the previous exercise, we saw the definition of arithmetic expressions with variables. 
The problem is that we do not know how to evaluate a variable, since the value of that is unknown.

In [1]:
type expr = 
	| Float of float
	| Var of string 
	| Neg of expr
	| Add of expr * expr
	| Mult of expr * expr

let rec eval = function
    | Float f -> f
    | Neg e -> -. (eval e)
    | Add (e1, e2) -> (eval e1) +. (eval e2)
    | Mult (e1, e2) -> (eval e1) *. (eval e2)
    | Var x -> failwith "Cannot evaluate!"

type expr =
    Float of float
  | Var of string
  | Neg of expr
  | Add of expr * expr
  | Mult of expr * expr


val eval : expr -> float = <fun>


The solution is to define an evaluator based on some *assignment* of the variables: a list of string-float pairs.
We include one more parameter `env` of type `(string * float) list`.
If `("x", 3.0)` is found in `env`, it means that `x` is assigned to the value `3.0`, and we evaluate variables 
by finding the associated value in the environment.

In [2]:
let rec eval env = function
    | Float f -> f
    | Neg e -> -. (eval env e)
    | Add (e1, e2) -> (eval env e1) +. (eval env e2)
    | Mult (e1, e2) -> (eval env e1) *. (eval env e2)
    | Var x -> List.assoc x env

val eval : (string * float) list -> expr -> float = <fun>


Note that the environment never changes. We can extend our language with constructs that modifies the environment, for example, by introducing bindings with `let` expressions.

In [ ]:
type expr = 
	| Float of float
	| Var of string 
	| Neg of expr
	| Add of expr * expr
	| Mult of expr * expr
	(* Extend let bindings *)
	| Let of string * expr * expr

type env = (string * float) list

To evaluate a let-binding, we need to update the environment with the variable-value pair.
The simplest way to do so is cons-ing the pair in front of `env`. Is there any major disadvantage in this method?

In [ ]:
(* interp : env -> expr -> float *)
let rec interp env = function
	| Float f -> f
	| Var x -> List.assoc x env
	| Neg e -> -1. *. interp env e
	| Add (e1 , e2) -> (interp env e1) +. (interp env e2)
	| Mult (e1, e2) -> (interp env e1) *. (interp env e2)
	(* Let-binding *)
	| Let (str, e1, e2) -> 
		let v = interp env e1 in
			interp ((str, v) :: env) e2

type expr =
    Float of float
  | Var of string
  | Neg of expr
  | Add of expr * expr
  | Mult of expr * expr
  | Let of string * expr * expr


type env = (string * float) list


val interp : (string * float) list -> expr -> float = <fun>


#### Programming with functions

We can now extend a bit further. Previously, the only value our expressions care about is float. We can make a new data type `value` to encorporate any kind of values we want to have in a *programming language*, which will include booleans, floats, and functions between values.

In [ ]:
type value = 
	| LitB of bool
	| LitF of float
	| Clo of (value -> value)

Starting with our definition of `expr` before, we can gradually extend it with new features:
- Boolean values and operations
- Functions and applications
- Recursive functions


**1. Booleans**

We need constructors for constants (`Bool`), if-branch (`If`), and boolean operators (`And`, `Or`, `Not`) for the boolean type,
and we add comparison (`Eq`) between values.

It also appears that we are duplicating code for `Add`, `Mult`, `And`, etc., and there should be an abstraction to encode all of these *binary operations* into one definition. We need two groups of them: one for the floating point numbers and one for the booleans

**2. Functions**

The syntax of a function `fun x -> e` is simple: it requires a variable `x` and an expression `e`, which is the function's body.
The function's body cannot be evaluated when the function is being evaluated, because the value of `x` (the input)
is not given yet. So, we interpret a function as an OCaml function that when given an input value, it evaluates the function body `e`
in the extended context. It also means that we can interpret applications in our language as applications in OCaml.

**3. Recursive functions**

There is a simple way to encode all recursive functions: the *fixed point combinator*, also known as the *Y-combinator*.
The technical detail here are irrelevant, but the key result is that: 
if we have a function `fix` such that `fix f x = f (fix f) x`, then all recursive definitions `let rec f x = e` can be
encoded as `fix (fun f x -> e)`, where the recursive function's name `f` is now treated as a variable in `e`.

It is certainly possible to take the condition `fix f x = f (fix f) x` as the definition
and implement the fixed point combinator in OCaml as a recursive function.
Then, we can interpret recursive functions in our language as recursive functions in OCaml.

To convince ourselves, let us see an example of the factorial function and its encoding in fixed point, and see if they produce the same result.
Our final example is the same factorial function defined in our own language.

In [ ]:
let rec fact n = if n = 0 then 1 else n * fact (n - 1)
let _ = fact 10

In [ ]:
let rec fix f x = f (fix f) x
let fact2 = fix (fun fact n -> if n = 0 then 1 else n * fact (n - 1)) 
let _ = fact2 10

In [ ]:
(* EXPRESSIONS *)

type expr = 
	(* Float *)
	| Float of float
	| Var of string 
	| Neg of expr
	| Add of expr * expr
	| Mult of expr * expr
	| Let of string * expr * expr
	(* Functions *)
	| Fun of string * expr
	| App of expr * expr
	(* Booleans *)
	| Bool of bool
	| If of expr * expr * expr
	| And of expr * expr
	| Or of expr * expr
	| Not of expr
	| Eq of expr * expr
	(* Recursion *)
	| RecFun of string * string * expr

exception TypeError

(* HELPER FUNCTIONS *)

let uop_f op = function
	| LitF f -> LitF (op f)
	| _ -> raise TypeError

let binop_f op v1 v2 = match (v1, v2) with
	| LitF f1 , LitF f2 -> LitF (op f1 f2)
	| _ -> raise TypeError

let rec fix f x = 
	f (fix f) x

let apply v1 v2 = match v1 with
	| Clo f -> f v2
	| _ -> raise TypeError

let uop_b op = function
	| LitB b -> LitB (op b)
	| _ -> raise TypeError

let binop_b op v1 v2 = match (v1, v2) with
	| LitB b1, LitB b2 -> LitB (op b1 b2)
	| _ -> raise TypeError

let eq v1 v2 = match (v1, v2) with
	| LitF f1, LitF f2 -> LitB (f1 = f2)
	| LitB b1, LitB b2 -> LitB (b1 = b2)
	| _ -> raise TypeError

let if_b = function
	| LitB b -> b
	| _ -> raise TypeError

(* INTERPRETER *)

let rec interp env = function
	| Float f -> LitF f
	| Var x -> List.assoc x env
	| Neg e -> uop_f (fun x -> -1. *. x) (interp env e)
	| Add (e1, e2) -> binop_f (+.) (interp env e1) (interp env e2)
	| Mult (e1, e2) -> binop_f ( *. ) (interp env e1) (interp env e2)
	| Let (str, e1, e2) -> 
		let v = interp env e1 in
			interp ((str, v) :: env) e2
	(* Function *)
	| Fun (str, e) -> Clo (fun v -> interp ((str, v) :: env) e)
	| App (e1, e2) -> apply (interp env e1) (interp env e2)
	(* Boolean *)
	| Bool b -> LitB b
	| And (e1, e2) -> binop_b (&&) (interp env e1) (interp env e2)
	| Or (e1, e2) -> binop_b (||) (interp env e1) (interp env e2)
	| Not e -> uop_b not (interp env e)
	| Eq (e1, e2) -> eq (interp env e1) (interp env e2)
	| If (e, e1, e2) -> 
		if if_b (interp env e) then 
			interp env e1
		else 
			interp env e2
	(* Recursive function *)
	| RecFun (name, str, e) -> 
			Clo (fix (fun f v -> interp ((name, Clo f) :: (str, v) :: env) e))

(* CODE EXAMPLE *)

let fact_lan = 
	RecFun("fact", "n", 
		If( 
			Eq(Var "n", Float 0.),
			Float 1.,
			Mult(Var "n", App(Var "fact", Add(Var "n", Float (-1.))))
		)
	)

let test = interp [] (App(fact_lan, Float 10.))

type value = LitB of bool | LitF of float | Clo of (value -> value)


type expr =
    Float of float
  | Var of string
  | Neg of expr
  | Add of expr * expr
  | Mult of expr * expr
  | Let of string * expr * expr
  | Fun of string * expr
  | App of expr * expr
  | Bool of bool
  | If of expr * expr * expr
  | And of expr * expr
  | Or of expr * expr
  | Not of expr
  | Eq of expr * expr
  | RecFun of string * string * expr


exception TypeError


val uop_f : (float -> float) -> value -> value = <fun>


val binop_f : (float -> float -> float) -> value -> value -> value = <fun>


...

val eq : value -> value -> value = <fun>


val if_b : value -> bool = <fun>


val interp : (string * value) list -> expr -> value = <fun>


val power : expr =
  RecFun ("power", "x",
   If (Eq (Var "x", Float 0.), Float 1.,
    Mult (Float 2., App (Var "power", Add (Var "x", Float (-1.))))))


val test : value = LitF 16.
